In [1]:
import open_clip
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_

In [4]:
from open_clip import create_model_and_transforms, get_tokenizer
import torch

device = torch.device("cuda")
tokenizer = get_tokenizer(model_name="ViT-L-14")
openai_clip, openai_preprocess_train, openai_preprocess_val = create_model_and_transforms(
    model_name="ViT-L-14",
    pretrained='openai',
    precision="bf16"
)
openai_clip = openai_clip.to(device)
    
datacomp_clip, datacomp_preprocess_train, datacomp_preprocess_val = create_model_and_transforms(
    model_name="ViT-L-14",
    pretrained='/home/user/data/LightClip/CLIP-KD/pretrained_models/CLIP-ViT-L-14-DataComp.XL-s13B-b90K/open_clip_pytorch_model.bin',
    precision="bf16"
)
datacomp_clip = datacomp_clip.to(device)

preprocess = openai_preprocess_val

In [2]:
openai_clip.logit_scale

Parameter containing:
tensor(4.6052, device='cuda:0', requires_grad=True)

In [3]:
datacomp_clip.logit_scale

Parameter containing:
tensor(4.6052, device='cuda:0', requires_grad=True)

In [5]:
from PIL import Image
image = Image.open('/home/user/data/test_cap.png')
img1 = openai_preprocess_val(image).type(torch.bfloat16).to(device).unsqueeze(0)
img2 = datacomp_preprocess_val(image).type(torch.bfloat16).to(device).unsqueeze(0)

text = tokenizer(["A Parent's Guide to Special Education logo.","a pretty girl in black", "a parent 's guide to special education ", "a parent 's guide to special education ", "a parent 's guide to special education "]).to(device)



In [6]:
text.shape

torch.Size([5, 77])

In [7]:
text[0]

tensor([49406,   320, 10183,   568,  3555,   531,  1689,  2806,  5750,   269,
        49407,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0], device='cuda:0')

In [4]:
output1 = openai_clip(img1,text)
output2 = datacomp_clip(img2,text)

In [5]:
img1f,t1f,logit_exp1 = output1
img2f,t2f,logit_exp2 = output2

In [6]:
img1f /= img1f.norm(dim=-1,keepdim=True)
img2f /= img2f.norm(dim=-1,keepdim=True)
t1f /= t1f.norm(dim=-1,keepdim=True)
t2f /= t2f.norm(dim=-1,keepdim=True)

In [7]:
logits1 = (100.0*img1f@t1f.T).softmax(dim=-1)

In [8]:
logits1

tensor([[2.3097e-07, 1.0000e+00, 1.7929e-04, 1.7929e-04, 1.7929e-04]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SoftmaxBackward0>)

In [9]:
import webdataset as wds
import torch
import io
from torch.nn.functional import cosine_similarity
dataset = wds.WebDataset("/home/user/data/cc12m_dr/00000.tar")
for sample in dataset:
    print(sample['paug.json'].decode('utf-8'))
    print(sample["syn_caps.json"].decode('utf-8'))
    print(sample["txt"].decode('utf-8'))
    buffer = io.BytesIO(sample['pth'])
    tensor = torch.load(buffer)
    print(tensor)
    print(cosine_similarity(tensor['text_emb'][2].unsqueeze(0),tensor['text_emb'][3].unsqueeze(0)))
    print(cosine_similarity(tensor['image_emb'][1].unsqueeze(0),tensor['image_emb'][3].unsqueeze(0)))
    break

[[[8, 36, 226, 207], [[3, 30.45317268371582], [8, -0.26999998092651367]]], [[152, 29, 102, 93], [[3, -30.45317268371582], [13, 0.0]]], [[81, 9, 134, 117], [[6, 0.26999998092651367], [4, -30.45317268371582]]], [[85, 18, 85, 84], [[9, 0.26999998092651367], [13, 0.0]]], [[9, 3, 236, 251], [[10, 7.0], [5, -9.0]]]]
["a man is holding a green object and a quote ", "a man holding a ball with a quote about it . ", "a man holding a tennis ball in a field . "]
The source of Anime quotes & Manga quotes : Photo <PERSON>, Manga Quotes, Art Images, Fan Art, Thoughts, Think, Anime, Crying, Random
{'image_emb': [tensor([ 1.1406, -0.1748,  0.2490,  ..., -1.0547,  1.8984,  0.5781],
       dtype=torch.bfloat16), tensor([ 0.6719, -0.1592, -1.0938,  ..., -0.9453,  1.4688,  0.8438],
       dtype=torch.bfloat16), tensor([ 0.7617,  0.1162, -0.6523,  ..., -0.5195,  1.6172,  0.5234],
       dtype=torch.bfloat16), tensor([ 0.2090,  0.1826, -0.2266,  ..., -0.3730,  3.7656,  0.2988],
       dtype=torch.bfloat16), 

In [10]:
import webdataset as wds
import torch
import io
from torch.nn.functional import cosine_similarity
dataset = wds.WebDataset("/home/user/data/cc12m_fixsyn_dr/00000.tar")
for sample in dataset:
    print(sample['paug.json'].decode('utf-8'))
    print(sample["syn_caps.json"].decode('utf-8'))
    print(sample["txt"].decode('utf-8'))
    buffer = io.BytesIO(sample['pth'])
    tensor = torch.load(buffer)
    print(tensor)
    print(cosine_similarity(tensor['text_emb'][2].unsqueeze(0),tensor['text_emb'][3].unsqueeze(0)))
    print(cosine_similarity(tensor['image_emb'][1].unsqueeze(0),tensor['image_emb'][3].unsqueeze(0)))
    break

[[[8, 36, 226, 207], [[3, 30.45317268371582], [8, -0.26999998092651367]]], [[152, 29, 102, 93], [[3, -30.45317268371582], [13, 0.0]]], [[81, 9, 134, 117], [[6, 0.26999998092651367], [4, -30.45317268371582]]], [[85, 18, 85, 84], [[9, 0.26999998092651367], [13, 0.0]]], [[9, 3, 236, 251], [[10, 7.0], [5, -9.0]]]]
["a man is holding a green object and a quote ", "a man holding a ball with a quote about it . ", "a man holding a tennis ball in a field . "]
The source of Anime quotes & Manga quotes : Photo <PERSON>, Manga Quotes, Art Images, Fan Art, Thoughts, Think, Anime, Crying, Random
{'image_emb': [tensor([ 1.1406, -0.1748,  0.2490,  ..., -1.0547,  1.8984,  0.5781],
       dtype=torch.bfloat16), tensor([ 0.6719, -0.1592, -1.0938,  ..., -0.9453,  1.4688,  0.8438],
       dtype=torch.bfloat16), tensor([ 0.7617,  0.1162, -0.6523,  ..., -0.5195,  1.6172,  0.5234],
       dtype=torch.bfloat16), tensor([ 0.2090,  0.1826, -0.2266,  ..., -0.3730,  3.7656,  0.2988],
       dtype=torch.bfloat16), 

In [11]:
import torch
tensor = torch.load('/home/user/data/cc12m_fixsyn_dr/00000/000009999.pth')

In [12]:
tensor

{'image_emb': [tensor([-0.0859,  0.6602,  0.6094,  ..., -0.6562,  0.4609,  1.2266],
         dtype=torch.bfloat16),
  tensor([ 0.5312,  0.3496,  0.8711,  ..., -1.0312,  2.4531,  0.1914],
         dtype=torch.bfloat16),
  tensor([-0.0371,  0.1074,  0.6367,  ..., -0.9180,  2.2188,  0.5078],
         dtype=torch.bfloat16),
  tensor([ 0.9062,  0.6133,  0.0977,  ..., -1.5703, -0.3867, -0.2715],
         dtype=torch.bfloat16),
  tensor([ 0.3359,  0.1641,  0.1748,  ..., -1.3359,  2.0156, -0.3594],
         dtype=torch.bfloat16)],
 'text_emb': [tensor([-0.5820, -0.3320, -0.0957,  ..., -0.5234,  0.2266, -1.5859],
         dtype=torch.bfloat16),
  tensor([ 0.2598, -0.0835,  0.4453,  ..., -0.6406,  1.8047, -0.5312],
         dtype=torch.bfloat16),
  tensor([-0.1377,  0.0830,  0.4062,  ..., -0.5000,  1.3672, -0.4707],
         dtype=torch.bfloat16),
  tensor([ 0.1406, -0.3301,  0.4219,  ..., -0.6211,  2.0781, -0.4102],
         dtype=torch.bfloat16)]}

In [13]:
len(tensor['image_emb'])

5

In [14]:
tensor['image_emb'][0].shape

torch.Size([1536])

In [15]:
len(tensor['text_emb'])

4

In [16]:
tensor['text_emb'][0].shape

torch.Size([1536])

In [17]:
import json
with open('/home/user/data/DataCompDR/DataCompDR-12M-bf16/00000000/5252820736bc680b29c6dca972776dc3.paug.json','r') as f:
    dic = json.load(f)
    

In [18]:
print(len(dic['param_aug']))

30


In [19]:
print(len(dic['param_aug'][0]))

2


In [20]:
print(dic['param_aug'][0])

[[33, 29, 48, 57], [[13, 0.0], [11, 178.5]]]


In [21]:
import gzip
import torch

with gzip.open('/home/user/data/DataCompDR/DataCompDR-12M-bf16/00000000/5252820736bc680b29c6dca972776dc3.pth.gz', 'rb') as f:
    tensor = torch.load(f)

In [22]:
print(len(tensor['text_emb']))

7


In [23]:
print(tensor['text_emb'][0].shape)

torch.Size([1536])


In [24]:
print(tensor['image_emb'][0].shape)

torch.Size([1536])


In [25]:
tensor['text_emb']

tensor([[ 0.0037, -0.0552, -0.0244,  ..., -0.0537, -0.0037,  0.0378],
        [ 0.0037, -0.0552, -0.0244,  ..., -0.0537, -0.0037,  0.0378],
        [ 0.0408, -0.0261,  0.0214,  ..., -0.0776, -0.0013, -0.0064],
        ...,
        [-0.0272, -0.0254,  0.0016,  ..., -0.0479,  0.0339,  0.0535],
        [-0.0109,  0.0195,  0.0005,  ..., -0.0520, -0.0298,  0.0072],
        [-0.0186, -0.0315, -0.0067,  ..., -0.0737,  0.0508,  0.0366]],
       dtype=torch.bfloat16)

In [26]:
tensor['text_emb'][0]

tensor([ 0.0037, -0.0552, -0.0244,  ..., -0.0537, -0.0037,  0.0378],
       dtype=torch.bfloat16)

In [27]:
tensor['text_emb'][1]

tensor([ 0.0037, -0.0552, -0.0244,  ..., -0.0537, -0.0037,  0.0378],
       dtype=torch.bfloat16)

In [28]:
tensor['text_emb'][2]

tensor([ 0.0408, -0.0261,  0.0214,  ..., -0.0776, -0.0013, -0.0064],
       dtype=torch.bfloat16)

In [29]:
tensor['text_emb'][0] == tensor['text_emb'][1]

tensor([True, True, True,  ..., True, True, True])

In [30]:
for i in (tensor['text_emb'][0] == tensor['text_emb'][1]):
    if i == True:
        continue
    else:
        print(False)
        break

In [31]:
torch.sum(tensor['text_emb'][0])

tensor(0.5859, dtype=torch.bfloat16)

In [32]:
torch.norm(torch.nn.functional.normalize(tensor['text_emb'][0],dim=-1))

tensor(1., dtype=torch.bfloat16)